# 0. The Obligatory Part

In [1]:
# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from competency_assessment import CompetencyAssessment
from ortools.sat.python import cp_model

# 1. Define the Data Structure

In [3]:
# Run this if the data in Local/Repository
new_employee_path = './mini_data/mini_data - employee.csv'
new_task_path = './mini_data/mini_data - task.csv'

## 1.1. Pre-Processing: Employee Data

In [4]:
# Read data
employee_skills_df = pd.read_csv(new_employee_path, index_col='employee_id')
employee_skills_df.drop(columns=['No', 'Role'], inplace=True, errors='ignore')

employees = employee_skills_df.index.tolist()
skills_name = employee_skills_df.columns[1:].tolist()

employee_skills_df

,math,python,sql,cloud,database,optimization
employee_id,,,,,,
Talent 1,5,5,3,2,2,5
Talent 2,2,5,4,3,3,1
Talent 3,4,4,5,3,4,3
Talent 4,3,4,5,5,4,2
Talent 5,2,3,2,4,5,1


## 1.2. Pre-Processing: Task Data

In [6]:
task_df = pd.read_csv(new_task_path, index_col='task_id')

tasks = task_df.index.tolist()
company_names = list(set(task_df['project_id']))
story_points = task_df['story_points'].to_dict()

task_df

,project_id,story_points,math,python,sql,cloud,database,optimization
task_id,,,,,,,,
T1,P2,1,0,5,3,4,4,0
T2,P1,2,3,5,4,5,3,1
T3,P2,3,5,4,2,0,0,5
T4,P3,5,2,4,0,3,0,4
T5,P3,1,0,3,5,1,5,0
T6,P1,1,0,1,4,5,5,0
T7,P3,2,4,4,1,1,2,3
T8,P1,3,1,0,5,5,3,0
T9,P2,1,3,4,3,0,0,2


## 1.3. Group the task data by company/project

In [7]:
# convert to dictionary each company and its task
company_tasks = {}

for company in company_names:
  company_tasks[company] = task_df[task_df['project_id'] == company].index.tolist()

# sort the company tasks from C1 to C5
company_tasks = dict(sorted(company_tasks.items()))

company_tasks_df = pd.DataFrame.from_dict(company_tasks, orient='index')
company_tasks_df

,0,1,2,3
P1,T2,T6,T8,T10
P2,T1,T3,T9,None
P3,T4,T5,T7,None


## 1.4. Pre-Processing: Competency Assesment

first, create RCD-ACD Dataframe

### 1.4.1 RCD Dataframe

In [10]:
rcd_df = task_df.drop(columns=['project_id', 'story_points'])
rcd_df

,math,python,sql,cloud,database,optimization
task_id,,,,,,
T1,0,5,3,4,4,0
T2,3,5,4,5,3,1
T3,5,4,2,0,0,5
T4,2,4,0,3,0,4
T5,0,3,5,1,5,0
T6,0,1,4,5,5,0
T7,4,4,1,1,2,3
T8,1,0,5,5,3,0
T9,3,4,3,0,0,2


### 1.4.2 ACD Dataframe

In [11]:
# create a copy of the original DataFrame
acd_df = employee_skills_df.copy()
acd_df

,math,python,sql,cloud,database,optimization
employee_id,,,,,,
Talent 1,5,5,3,2,2,5
Talent 2,2,5,4,3,3,1
Talent 3,4,4,5,3,4,3
Talent 4,3,4,5,5,4,2
Talent 5,2,3,2,4,5,1


### 1.4.3 Fit the Data

In [12]:
ca = CompetencyAssessment(rcd_df, acd_df)
qs, info = ca.fit()

In [14]:
qs

{'Talent 1': {'T1': [0.0, -1.0, -0.16666666666666666, 'Under-Qualified'],
  'T2': [0.4761904761904761,
   -1.0476190476190477,
   -0.09523809523809527,
   'Under-Qualified'],
  'T3': [0.375, 0, 0.0625, 'Qualified'],
  'T4': [1.076923076923077,
   -0.23076923076923073,
   0.14102564102564105,
   'Qualified'],
  'T5': [0.5, -1.785714285714286, -0.21428571428571433, 'Under-Qualified'],
  'T6': [0.26666666666666666,
   -2.2666666666666666,
   -0.3333333333333333,
   'Under-Qualified'],
  'T7': [1.133333333333333, 0, 0.18888888888888886, 'Qualified'],
  'T8': [0.2857142857142857,
   -2.0000000000000004,
   -0.2857142857142858,
   'Under-Qualified'],
  'T9': [1.3333333333333333, 0, 0.2222222222222222, 'Qualified'],
  'T10': [0.0, 0, 0.0, 'Qualified']},
 'Talent 2': {'T1': [0.1875, -0.5, -0.052083333333333336, 'Under-Qualified'],
  'T2': [0.0, -0.6190476190476191, -0.10317460317460318, 'Under-Qualified'],
  'T3': [0.5, -2.1875, -0.28125, 'Under-Qualified'],
  'T4': [0.3076923076923077,
   -0.

In [15]:
score = ca.rank_MSG(qs)
score

{'Talent 1': {'T9': 0.2222222222222222,
  'T7': 0.18888888888888886,
  'T4': 0.14102564102564105,
  'T3': 0.0625,
  'T10': 0.0,
  'T2': -0.09523809523809527,
  'T1': -0.16666666666666666,
  'T5': -0.21428571428571433,
  'T8': -0.2857142857142858,
  'T6': -0.3333333333333333},
 'Talent 2': {'T9': 0.027777777777777773,
  'T7': -0.033333333333333326,
  'T1': -0.052083333333333336,
  'T5': -0.08333333333333337,
  'T4': -0.10256410256410257,
  'T2': -0.10317460317460318,
  'T8': -0.1666666666666667,
  'T6': -0.17777777777777773,
  'T3': -0.28125,
  'T10': -0.38888888888888884},
 'Talent 3': {'T9': 0.1527777777777778,
  'T7': 0.11111111111111112,
  'T4': 0.0,
  'T5': -9.25185853854297e-18,
  'T2': -0.023809523809523836,
  'T1': -0.03125,
  'T8': -0.04761904761904764,
  'T6': -0.08888888888888886,
  'T3': -0.09375,
  'T10': -0.22222222222222218},
 'Talent 4': {'T9': 0.08333333333333333,
  'T8': 0.05952380952380953,
  'T7': 0.05555555555555556,
  'T1': 0.052083333333333336,
  'T2': 0.023809523

In [17]:
all_top_n_scores = ca.all_top_n_score(score, 50)
all_top_n_scores

{'Talent 1': {'T9': 0.2222222222222222,
  'T7': 0.18888888888888886,
  'T4': 0.14102564102564105,
  'T3': 0.0625,
  'T10': 0.0},
 'Talent 2': {'T9': 0.027777777777777773,
  'T7': -0.033333333333333326,
  'T1': -0.052083333333333336,
  'T5': -0.08333333333333337,
  'T4': -0.10256410256410257},
 'Talent 3': {'T9': 0.1527777777777778,
  'T7': 0.11111111111111112,
  'T4': 0.0,
  'T5': -9.25185853854297e-18,
  'T2': -0.023809523809523836},
 'Talent 4': {'T9': 0.08333333333333333,
  'T8': 0.05952380952380953,
  'T7': 0.05555555555555556,
  'T1': 0.052083333333333336,
  'T2': 0.023809523809523798},
 'Talent 5': {'T7': -0.08888888888888889,
  'T1': -0.09375,
  'T6': -0.12222222222222219,
  'T5': -0.14285714285714288,
  'T8': -0.1547619047619048}}